## Thuật toán xây dựng đa thức nội suy bằng phương pháp Bessel 

In [11]:
import numpy as np
import pandas as pd
import math

def load_and_validate_bessel_data(file_x_path, file_y_path):
    """
    Hàm đọc dữ liệu từ file và kiểm tra các điều kiện cho nội suy Bessel.
    """
    try:
        x_coords = np.loadtxt(file_x_path, ndmin=1).astype(float)
        y_coords = np.loadtxt(file_y_path, ndmin=1).astype(float)
    except (FileNotFoundError, ValueError):
        raise IOError("LỖI: Không tìm thấy file hoặc file chứa dữ liệu không hợp lệ.")

    if len(x_coords) != len(y_coords):
        raise ValueError("LỖI: Số lượng điểm x và y không khớp.")

    # >>> KIỂM TRA MỚI: Số lượng điểm phải là chẵn <<<
    if len(x_coords) % 2 != 0:
        raise ValueError("LỖI: Công thức Bessel yêu cầu số lượng mốc nội suy phải là số chẵn (2m).")

    if len(np.unique(x_coords)) != len(x_coords):
        raise ValueError("LỖI: Các giá trị x không được trùng lặp.")
        
    # Kiểm tra điều kiện quan trọng: các mốc x phải cách đều
    diffs = np.diff(x_coords)
    # Tăng độ chính xác khi so sánh
    if not np.allclose(diffs, diffs[0], rtol=1e-08, atol=1e-10): 
        raise ValueError("LỖI: Các mốc nội suy x phải cách đều nhau để áp dụng công thức Bessel.")
        
    return x_coords, y_coords

# --- Chương trình chính cho Ô 1 ---
# Đảm bảo file FileX.txt và FileY.txt chứa số lượng điểm chẵn
file_x = 'FileX.txt'
file_y = 'FileY.txt'

try:
    x_coords, y_coords = load_and_validate_bessel_data(file_x, file_y)
    print("Dữ liệu hợp lệ cho công thức Bessel và đã được đọc thành công!")
    print("-" * 50)
    print("Các mốc nội suy x:", x_coords.tolist())
    print("Các giá trị y tương ứng:", y_coords.tolist())
except (IOError, ValueError) as e:
    print(str(e))

Dữ liệu hợp lệ cho công thức Bessel và đã được đọc thành công!
--------------------------------------------------
Các mốc nội suy x: [5.0, 5.1, 5.2, 5.3, 5.4, 5.5, 5.6, 5.7]
Các giá trị y tương ứng: [3.9931011, 4.1278568, 4.2723258, 4.426064, 4.5885338, 4.7591118, 4.93709, 5.1216967]


In [12]:
# Giả định x_coords và y_coords đã tồn tại từ Ô 1

# --- Hàm tạo bảng sai phân đầy đủ (dùng cho lấy giá trị) ---
def create_full_difference_table(y_values):
    num_points = len(y_values); table = [[0.0 for _ in range(num_points)] for _ in range(num_points)]
    for i in range(num_points): table[i][0] = y_values[i]
    for j in range(1, num_points):
        for i in range(num_points - j): table[i][j] = table[i+1][j-1] - table[i][j-1]
    return np.array(table)

# --- Hàm tạo DataFrame hiển thị dạng tam giác dưới ---
def create_lower_diagonal_df(x_values, y_values):
    num_points = len(y_values)
    # Xác định chỉ số của x_0 (ngay trước khoảng giữa) cho mục đích hiển thị
    center_idx_display = (num_points // 2) - 1
    table = [[np.nan for _ in range(num_points)] for _ in range(num_points)]
    for i in range(num_points): table[i][0] = y_values[i]
    for j in range(1, num_points):
        for i in range(j, num_points): table[i][j] = table[i][j-1] - table[i-1][j-1]
    headers = ['y'] + [f'SP{i}' for i in range(1, num_points)]; df = pd.DataFrame(table, columns=headers)
    df.insert(0, 'x', x_values); positions = [f'x_{i - center_idx_display}' for i in range(num_points)]; df.insert(0, 'Vị trí', positions)
    return df

# Tạo các bảng cần thiết
diff_table_for_calc = create_full_difference_table(y_coords)
df_display = create_lower_diagonal_df(x_coords, y_coords)

# --- Xác định tọa độ các giá trị trung tâm CẦN TÔ MÀU ---
# Với số điểm chẵn (n), ta quan tâm đến các giá trị quanh chỉ số n/2 - 1 và n/2
coords_to_highlight = set()
center_idx_calc = (len(x_coords) // 2) - 1 # Chỉ số của hàng y_0 trong bảng tính

# 1. Thêm y_0 và y_1 (dùng center_idx_calc làm chỉ số cho y_0)
coords_to_highlight.add((center_idx_calc, 0)) # y_0
coords_to_highlight.add((center_idx_calc + 1, 0)) # y_1

# 2. Thêm các sai phân trung tâm
for j in range(1, len(x_coords)): # j là bậc sai phân
    # Chỉ số hàng trên và dưới quanh đường trung tâm trong bảng tính
    row_upper = center_idx_calc - (j // 2)
    row_lower = center_idx_calc - ((j - 1) // 2)
    
    # Đảm bảo chỉ số hàng không vượt quá giới hạn bảng
    if row_upper >= 0:
        coords_to_highlight.add((row_upper, j))
    if row_lower < len(x_coords) - j: # Kiểm tra giới hạn dưới của bảng tính
         coords_to_highlight.add((row_lower, j))


# --- Tạo hàm style dựa trên tọa độ ---
def highlight_central_values(df_slice):
    style = 'background-color: yellow; font-weight: bold; color: black;'
    style_df = pd.DataFrame('', index=df_slice.index, columns=df_slice.columns)
    
    # Ánh xạ tọa độ từ bảng tính toán sang bảng hiển thị tam giác dưới
    for r_calc, c_calc in coords_to_highlight:
        # Tên cột trên bảng hiển thị
        c_display_name = f'SP{c_calc}' if c_calc > 0 else 'y'
        # Chỉ số hàng trên bảng hiển thị
        r_display = r_calc + c_calc
        
        # Kiểm tra và tô màu
        if c_display_name in style_df.columns and r_display in style_df.index:
            val = df_slice.loc[r_display, c_display_name]
            if not pd.isna(val): # Chỉ tô màu ô có giá trị
                 style_df.loc[r_display, c_display_name] = style
            
    return style_df

# --- Áp dụng định dạng và hiển thị (không làm tròn) ---
print(f"\n--- Bảng sai phân (làm nổi bật các giá trị trung tâm) ---")
numeric_cols = df_display.select_dtypes(include=np.number).columns
styled_df = df_display.style.apply(highlight_central_values, axis=None).format(
    "{:g}", 
    subset=numeric_cols, 
    na_rep=""
)

display(styled_df)


--- Bảng sai phân (làm nổi bật các giá trị trung tâm) ---


,Vị trí,x,y,SP1,SP2,SP3,SP4,SP5,SP6,SP7
0,x_-3,5,3.9931,,,,,,,
1,x_-2,5.1,4.12786,0.134756,,,,,,
2,x_-1,5.2,4.27233,0.144469,0.0097133,,,,,
3,x_0,5.3,4.42606,0.153738,0.0092692,-0.0004441,,,,
4,x_1,5.4,4.58853,0.16247,0.0087316,-0.0005376,-9.35e-05,,,
5,x_2,5.5,4.75911,0.170578,0.0081082,-0.0006234,-8.58e-05,7.7e-06,,
6,x_3,5.6,4.93709,0.177978,0.0074002,-0.000708,-8.46e-05,1.2e-06,-6.5e-06,
7,x_4,5.7,5.1217,0.184607,0.0066285,-0.0007717,-6.37e-05,2.09e-05,1.97e-05,2.62e-05


In [13]:
# Giả định các biến sau đã tồn tại từ các ô trước:
# x_coords: Mảng NumPy chứa các mốc nội suy x.
# y_coords: Mảng NumPy chứa các giá trị y.
# diff_table_for_calc: Ma trận NumPy chứa bảng sai phân đầy đủ (dạng tam giác trên).

def calculate_bessel_coeffs(x_data, full_diff_table):
    """
    Tính toán vector hệ số (tham số) cho công thức Bessel.
    """
    n = len(x_data)
    # Chỉ số của x0 (điểm ngay trước khoảng giữa)
    center_idx = (n // 2) - 1
    
    bessel_coeffs = []
    
    # Lặp qua các bậc sai phân từ 0 đến n-1
    for k in range(n):
        if k == 0:
            # Bậc 0: Trung bình cộng của y0 và y1, chia 0! = 1
            y0 = full_diff_table[center_idx, 0]
            y1 = full_diff_table[center_idx + 1, 0]
            coeff = (y0 + y1) / 2.0
        elif k % 2 != 0: # Bậc lẻ: 1, 3, 5, ...
            # Lấy giá trị sai phân Δ^k * y_(center_idx - (k-1)//2)
            # Ví dụ k=1 -> Δ^1*y_0 ; k=3 -> Δ^3*y_-1 ; k=5 -> Δ^5*y_-2
            row_idx = center_idx - ((k - 1) // 2)
            # Kiểm tra chỉ số hàng hợp lệ
            if row_idx >= 0 and row_idx < (n - k):
                 diff_val = full_diff_table[row_idx, k]
                 coeff = diff_val / math.factorial(k)
            else:
                 coeff = 0.0 # Hoặc NaN nếu muốn chỉ ra lỗi
        else: # Bậc chẵn: 2, 4, 6, ...
            # Lấy trung bình cộng của Δ^k * y_(center_idx - k//2) và Δ^k * y_(center_idx - k//2 + 1)
            # Ví dụ k=2 -> (Δ^2*y_-1 + Δ^2*y_0)/2 ; k=4 -> (Δ^4*y_-2 + Δ^4*y_-1)/2
            row_idx_upper = center_idx - (k // 2)
            row_idx_lower = row_idx_upper + 1
            # Kiểm tra chỉ số hàng hợp lệ
            if row_idx_upper >= 0 and row_idx_lower < (n-k+1) :
                 diff1 = full_diff_table[row_idx_upper, k]
                 diff2 = full_diff_table[row_idx_lower, k]
                 avg_diff = (diff1 + diff2) / 2.0
                 coeff = avg_diff / math.factorial(k)
            elif row_idx_upper >= 0 and row_idx_upper < (n-k):
                 # Trường hợp chỉ có 1 giá trị ở biên
                 diff1 = full_diff_table[row_idx_upper, k]
                 coeff = diff1 / (2.0 * math.factorial(k)) # Chia 2? Cần kiểm tra lại định nghĩa Bessel ở biên
                 coeff = diff1 / math.factorial(k) # Theo công thức Bessel chuẩn
            else:
                 coeff = 0.0 # Hoặc NaN
                 
        bessel_coeffs.append(coeff)
        
    return np.array(bessel_coeffs)

# --- Thực thi và in kết quả ---
bessel_parameter_vector = calculate_bessel_coeffs(x_coords, diff_table_for_calc)

print("--- Vector Hệ số Bessel (Đã chia giai thừa) ---")
# Sử dụng printoptions để hiển thị đầy đủ
with np.printoptions(precision=15, suppress=True):
    print(bessel_parameter_vector)

# (Tùy chọn) In vector chẵn/lẻ riêng nếu muốn
even_bessel_coeffs = bessel_parameter_vector[::2]
odd_bessel_coeffs = bessel_parameter_vector[1::2]
print("\nPhần chẵn:", even_bessel_coeffs)
print("Phần lẻ:", odd_bessel_coeffs)

--- Vector Hệ số Bessel (Đã chia giai thừa) ---
[ 4.5072989          0.1624698          0.00420995
 -0.0001039         -0.00000355         0.00000001
  0.000000009166667  0.000000005198413]

Phần chẵn: [ 4.50729890e+00  4.20995000e-03 -3.55000000e-06  9.16666666e-09]
Phần lẻ: [ 1.6246980e-01 -1.0390000e-04  1.0000000e-08  5.1984127e-09]


In [14]:
# Giả định các biến sau đã tồn tại từ các ô trước:
# odd_coeffs: Vector hệ số lẻ (đã chia giai thừa)

# --- Bước 1: Chuẩn bị dữ liệu ---
n_odd = len(odd_bessel_coeffs) # Số lượng hệ số lẻ
num_rows_product_table_odd = n_odd - 1 # Số hàng tính toán

# Tính các giá trị c = (2k+1)^2 / 4
c_values_odd = [((2 * k + 1)**2) / 4.0 for k in range(num_rows_product_table_odd)]

print(f"Số lượng hệ số lẻ: {n_odd}")
print(f"Số hàng sẽ tính trong bảng tích: {num_rows_product_table_odd}")
print(f"Các giá trị c = (2k+1)^2 / 4: {c_values_odd}")

# --- Bước 2: Tạo Bảng Tích ---

# Khởi tạo bảng với NaN, kích thước (số hàng tính + 1) x (số hệ số lẻ)
product_table_data_odd = np.full((num_rows_product_table_odd + 1, n_odd), np.nan)

# Hàng đầu tiên (hàng 0) là [0, 0, ..., 1]
product_table_data_odd[0, :] = 0.0
product_table_data_odd[0, -1] = 1.0

# Lặp để tính các hàng tiếp theo
# a: là hàng trước đó
# b: là hàng đang tính
for row_idx in range(num_rows_product_table_odd):
    c = c_values_odd[row_idx]
    a = product_table_data_odd[row_idx] # Lấy hàng đã tính ở bước trước
    b = np.zeros(n_odd)             # Hàng mới

    # Áp dụng công thức b_k = a_{k+1} - a_k * c
    for k in range(n_odd):
        # Lấy a[k+1], nếu k là chỉ số cuối cùng thì coi như là 0
        a_k_plus_1 = a[k+1] if k < n_odd - 1 else 0.0 
        a_k = a[k]
        b[k] = a_k_plus_1 - (a_k * c) # Tính giá trị b_k
        
    # Lưu hàng mới vào bảng
    product_table_data_odd[row_idx + 1] = b

# --- Bước 3: Tạo và hiển thị DataFrame (Đã sửa lỗi) ---

# Tạo header cho các cột tính toán
product_headers_odd = [f'Col_{i}' for i in range(n_odd)]

# >>> THAY ĐỔI Ở ĐÂY: Tạo danh sách index <<<
product_index_list = ['Khởi tạo (a)'] + [f'k={k}' for k in range(num_rows_product_table_odd)]

# >>> THAY ĐỔI Ở ĐÂY: Truyền index vào lúc tạo DataFrame <<<
# Tạo DataFrame từ dữ liệu tính toán, đồng thời gán index
df_product_odd = pd.DataFrame(
    product_table_data_odd,
    columns=product_headers_odd,
    index=product_index_list # Truyền danh sách index vào đây
)

# Thêm cột giá trị 'c' vào đầu
c_column_display = [np.nan] + c_values_odd
df_product_odd.insert(0, 'Giá trị c ((2k+1)^2/4)', c_column_display)

# Đặt tên cho index (thay vì gán trực tiếp)
df_product_odd.index.name = "Bước tính (k)" # Đặt tên cho cột index

# Định dạng hiển thị (không làm tròn)
styled_product_df_odd = df_product_odd.style.format("{:g}", na_rep="")

print("\n--- Bảng Tích ---")
display(styled_product_df_odd)

Số lượng hệ số lẻ: 4
Số hàng sẽ tính trong bảng tích: 3
Các giá trị c = (2k+1)^2 / 4: [0.25, 2.25, 6.25]

--- Bảng Tích ---


,Giá trị c ((2k+1)^2/4),Col_0,Col_1,Col_2,Col_3
Bước tính (k),,,,,
Khởi tạo (a),,0,0,0,1
k=0,0.25,0,0,1,-0.25
k=1,2.25,0,1,-2.5,0.5625
k=2,6.25,1,-8.75,16.1875,-3.51562


In [15]:
# Giả định các biến sau đã tồn tại từ các ô trước:
# even_coeffs (hoặc even_bessel_coeffs): Vector hệ số chẵn ban đầu
# odd_coeffs (hoặc odd_bessel_coeffs): Vector hệ số lẻ ban đầu
# product_table_data_odd: Ma trận NumPy chứa dữ liệu của Bảng Tích LẺ

# --- B1: In lại vector tham số chẵn và lẻ ban đầu ---
print("--- B1: Vector tham số chẵn và lẻ ban đầu ---")
# Đảm bảo sử dụng đúng tên biến bạn đã định nghĩa
print("Vector chẵn:")
if 'even_bessel_coeffs' in locals():
    print(even_bessel_coeffs)
    even_vector_to_use = even_bessel_coeffs
else:
    print("LỖI: Không tìm thấy vector hệ số chẵn.")
    even_vector_to_use = None

print("\nVector lẻ:")
if 'odd_bessel_coeffs' in locals():
    print(odd_bessel_coeffs)
    odd_vector_to_use = odd_bessel_coeffs
else:
    print("LỖI: Không tìm thấy vector hệ số lẻ.")
    odd_vector_to_use = None
print("-" * 50)

# --- B2: In ra ma trận DUY NHẤT sẽ được sử dụng (Bảng Tích LẺ) ---
print("--- B2: Ma trận sẽ dùng để nhân (Bảng Tích LẺ) ---")
# >>> THAY ĐỔI Ở ĐÂY: Sử dụng product_table_data_odd <<<
if 'product_table_data_odd' in locals():
    matrix_to_use = product_table_data_odd # Gán ma trận lẻ vào biến dùng chung
    with np.printoptions(precision=15, suppress=True, threshold=np.inf):
        print(matrix_to_use)
else:
    print("LỖI: Biến 'product_table_data_odd' chưa được định nghĩa.")
    matrix_to_use = None
print("-" * 50)

# --- B3: Thực hiện phép nhân và GÁN KẾT QUẢ ---
print("--- B3: Thực hiện phép nhân ---")
new_even_coeffs = None
new_odd_coeffs = None

# Phép nhân cho vector chẵn (SỬ DỤNG MA TRẬN LẺ)
if matrix_to_use is not None and even_vector_to_use is not None:
    print("Tính vector chẵn mới:")
    try:
        # >>> SỬ DỤNG CÙNG MA TRẬN matrix_to_use <<<
        print(f"Thực hiện phép nhân: even_vector_to_use @ matrix_to_use")
        new_even_coeffs = even_vector_to_use @ matrix_to_use # Gán kết quả
        print("Kết quả vector chẵn mới:")
        print("[", end="")
        for i, val in enumerate(new_even_coeffs):
            print(f"{val}", end="")
            if i < len(new_even_coeffs) - 1: print(", ", end="")
        print("]")
    except ValueError as e:
        print(f"Lỗi khi nhân ma trận cho vector chẵn: {e}.")
        print(f"  Kích thước vector chẵn: {even_vector_to_use.shape}")
        print(f"  Kích thước ma trận: {matrix_to_use.shape}")
        print("LƯU Ý: Kích thước có thể không khớp.")
else:
    print("Không thể tính vector chẵn mới (thiếu ma trận hoặc vector ban đầu).")

print("-" * 30)

# Phép nhân cho vector lẻ (SỬ DỤNG CÙNG MA TRẬN)
if matrix_to_use is not None and odd_vector_to_use is not None:
    print("Tính vector lẻ mới:")
    try:
        # >>> SỬ DỤNG CÙNG MA TRẬN matrix_to_use <<<
        print(f"Thực hiện phép nhân: odd_vector_to_use @ matrix_to_use")
        new_odd_coeffs = odd_vector_to_use @ matrix_to_use # Gán kết quả
        print("Kết quả vector lẻ mới:")
        print("[", end="")
        for i, val in enumerate(new_odd_coeffs):
            print(f"{val}", end="")
            if i < len(new_odd_coeffs) - 1: print(", ", end="")
        print("]")
    except ValueError as e:
        print(f"Lỗi khi nhân ma trận cho vector lẻ: {e}.")
        print(f"  Kích thước vector lẻ: {odd_vector_to_use.shape}")
        print(f"  Kích thước ma trận: {matrix_to_use.shape}")
        print("LƯU Ý: Kích thước có thể không khớp.")
else:
    print("Không thể tính vector lẻ mới (thiếu ma trận hoặc vector ban đầu).")

print("=" * 50)
print("--- KẾT THÚC Ô TÍNH TOÁN ---")

# --- Ô tiếp theo: Kết hợp vector ---
# Ô code kết hợp vector sẽ sử dụng new_even_coeffs và new_odd_coeffs vừa tính.

--- B1: Vector tham số chẵn và lẻ ban đầu ---
Vector chẵn:
[ 4.50729890e+00  4.20995000e-03 -3.55000000e-06  9.16666666e-09]

Vector lẻ:
[ 1.6246980e-01 -1.0390000e-04  1.0000000e-08  5.1984127e-09]
--------------------------------------------------
--- B2: Ma trận sẽ dùng để nhân (Bảng Tích LẺ) ---
[[ 0.        0.        0.        1.      ]
 [ 0.        0.        1.       -0.25    ]
 [ 0.        1.       -2.5       0.5625  ]
 [ 1.       -8.75     16.1875   -3.515625]]
--------------------------------------------------
--- B3: Thực hiện phép nhân ---
Tính vector chẵn mới:
Thực hiện phép nhân: even_vector_to_use @ matrix_to_use
Kết quả vector chẵn mới:
[9.16666666492569e-09, -3.6302083332999086e-06, 0.0042189733854165, 4.506244383398438]
------------------------------
Tính vector lẻ mới:
Thực hiện phép nhân: odd_vector_to_use @ matrix_to_use
Kết quả vector lẻ mới:
[5.1984126979233636e-09, -3.5486111100990756e-08, -0.00010384085069451259, 0.16249576234933058]
--- KẾT THÚC Ô TÍNH TOÁN ---

In [16]:
# Giả định các biến sau đã tồn tại từ Ô TRÊN:
# new_even_coeffs: Vector hệ số chẵn mới (kết quả phép nhân)
# new_odd_coeffs: Vector hệ số lẻ mới (kết quả phép nhân)
# even_coeffs: Vector hệ số chẵn BAN ĐẦU (để lấy độ dài)
# odd_coeffs: Vector hệ số lẻ BAN ĐẦU (để lấy độ dài)

print("--- Bắt đầu kết hợp vector hệ số ---")
print("Sử dụng các vector mới đã tính:")
if new_even_coeffs is not None:
    with np.printoptions(threshold=np.inf): print("Vector chẵn mới:", new_even_coeffs)
else: print("Vector chẵn mới: Lỗi!")
if new_odd_coeffs is not None:
    with np.printoptions(threshold=np.inf): print("Vector lẻ mới:", new_odd_coeffs)
else: print("Vector lẻ mới: Lỗi!")
print("-" * 30)


# Chỉ thực hiện kết hợp nếu cả hai vector mới đều hợp lệ
if new_even_coeffs is not None and new_odd_coeffs is not None:

    # Xác định độ dài và bậc dựa trên vector ban đầu
    len_even_orig = len(even_bessel_coeffs)
    len_odd_orig = len(odd_bessel_coeffs)
    total_coeffs = len_even_orig + len_odd_orig
    
    # Khởi tạo vector kết quả cuối cùng (theo bậc giảm dần)
    final_coeffs_descending = np.zeros(total_coeffs)

    even_idx = 0
    odd_idx = 0

    # Lặp từ đầu (bậc cao) đến cuối (bậc thấp) của final_coeffs_descending
    for i in range(total_coeffs):
        power = total_coeffs - 1 - i # Bậc hiện tại
        if power % 2 == 0: # Bậc chẵn
            if even_idx < len(new_even_coeffs):
                final_coeffs_descending[i] = new_even_coeffs[even_idx]
                even_idx += 1
        else: # Bậc lẻ
            if odd_idx < len(new_odd_coeffs):
                final_coeffs_descending[i] = new_odd_coeffs[odd_idx]
                odd_idx += 1

    # >>> ĐỊNH NGHĨA BIẾN parameter_vector_coeffs <<<
    # Gán kết quả cuối cùng (đang ở bậc giảm dần)
    parameter_vector_coeffs_desc = final_coeffs_descending
    # Hoặc nếu bạn muốn nó theo bậc tăng dần:
    parameter_vector_coeffs = final_coeffs_descending[::-1]


    # --- In kết quả ---
    print("--- Vector hệ số cuối cùng (ĐTNS) đã được kết hợp ---")
    print("(Sắp xếp theo bậc giảm dần - parameter_vector_coeffs_desc):")
    with np.printoptions(threshold=np.inf):
        print(parameter_vector_coeffs_desc)

    # (Tùy chọn) In ra đa thức dạng chuỗi

    def format_polynomial_descending(coeffs):
        """
        Định dạng đa thức từ vector hệ số (bậc tăng dần).
        Trả về chuỗi biểu diễn đa thức với bậc giảm dần.
        """
        terms = []
        n = len(coeffs)
        for i, coef in enumerate(reversed(coeffs)):
            deg = n - 1 - i
            if abs(coef) < 1e-14:
                continue
            # Định dạng hệ số
            if deg == 0:
                term = f"{coef:.10g}"
            elif deg == 1:
                if coef == 1:
                    term = "u"
                elif coef == -1:
                    term = "-u"
                else:
                    term = f"{coef:.10g}*u"
            else:
                if coef == 1:
                    term = f"u^{deg}"
                elif coef == -1:
                    term = f"-u^{deg}"
                else:
                    term = f"{coef:.10g}*u^{deg}"
            terms.append(term)
        if not terms:
            return "0"
        return " + ".join(terms).replace("+ -", "- ")

    print("\n--- Đa thức nội suy P(u) hoàn chỉnh ---")
    print(f"P(x) =", format_polynomial_descending(parameter_vector_coeffs))

else:
    print("LỖI: Không thể kết hợp vector hệ số do lỗi trong các bước tính toán trước đó.")

print("=" * 50)
print("--- KẾT THÚC ---")

--- Bắt đầu kết hợp vector hệ số ---
Sử dụng các vector mới đã tính:
Vector chẵn mới: [ 9.16666666e-09 -3.63020833e-06  4.21897339e-03  4.50624438e+00]
Vector lẻ mới: [ 5.19841270e-09 -3.54861111e-08 -1.03840851e-04  1.62495762e-01]
------------------------------
--- Vector hệ số cuối cùng (ĐTNS) đã được kết hợp ---
(Sắp xếp theo bậc giảm dần - parameter_vector_coeffs_desc):
[ 5.19841270e-09  9.16666666e-09 -3.54861111e-08 -3.63020833e-06
 -1.03840851e-04  4.21897339e-03  1.62495762e-01  4.50624438e+00]

--- Đa thức nội suy P(u) hoàn chỉnh ---
P(x) = 5.198412698e-09*u^7 + 9.166666665e-09*u^6 - 3.54861111e-08*u^5 - 3.630208333e-06*u^4 - 0.0001038408507*u^3 + 0.004218973385*u^2 + 0.1624957623*u + 4.506244383
--- KẾT THÚC ---


## Đoạn code tính đạo hàm của P^(n)(x) tại x = x'


In [17]:
from numpy.polynomial import polynomial as P

# Giả định x_coords và diff_table_for_calc đã tồn tại

def get_bessel_coeffs_in_u(x_data, full_diff_table):
    """Tính các hệ số của đa thức P(u) theo công thức Bessel."""
    n = len(x_data)
    center_idx = (n // 2) - 1 # Chỉ số của y0
    
    # --- Số hạng 0: (y0 + y1)/2 ---
    y0 = full_diff_table[center_idx, 0]
    y1 = full_diff_table[center_idx + 1, 0]
    final_poly_in_u = np.array([(y0 + y1) / 2.0])
    
    # --- Số hạng 1: u * Δy0 ---
    delta_y0 = full_diff_table[center_idx, 1]
    term1_poly = np.array([0, delta_y0]) # Hệ số của delta_y0 * u
    final_poly_in_u = P.polyadd(final_poly_in_u, term1_poly)
    
    # --- Các số hạng bậc cao hơn ---
    product_poly_u2 = np.array([1.0]) # Bắt đầu với 1
    
    for i in range(1, n // 2): 
        # Cập nhật đa thức tích (u^2 - k^2)
        k_sq = ((2*i - 1) / 2)**2
        factor_poly = np.array([-k_sq, 0, 1.0]) # u^2 - k^2
        product_poly_u2 = P.polymul(product_poly_u2, factor_poly)
        
        # --- Số hạng bậc chẵn 2i ---
        term_order_even = 2*i
        if term_order_even >= n: break
        diff1 = full_diff_table[center_idx - i, term_order_even]
        diff2 = full_diff_table[center_idx - i + 1, term_order_even]
        avg_diff_even = (diff1 + diff2) / 2.0
        term_even_poly = (avg_diff_even / math.factorial(term_order_even)) * product_poly_u2
        final_poly_in_u = P.polyadd(final_poly_in_u, term_even_poly)
        
        # --- Số hạng bậc lẻ 2i+1 ---
        term_order_odd = 2*i + 1
        if term_order_odd >= n: break
        diff_odd = full_diff_table[center_idx - i, term_order_odd]
        u_poly = np.array([0, 1.0]) # Đa thức u
        u_times_product = P.polymul(u_poly, product_poly_u2)
        term_odd_poly = (diff_odd / math.factorial(term_order_odd)) * u_times_product
        final_poly_in_u = P.polyadd(final_poly_in_u, term_odd_poly)
        
    return final_poly_in_u

# --- Thực thi và định nghĩa biến coeffs_in_u ---
coeffs_in_u = get_bessel_coeffs_in_u(x_coords, diff_table_for_calc)

print("--- Các hệ số của đa thức theo ẩn u, P(u) [Bessel] ---")
with np.printoptions(precision=15, suppress=True):
    print(coeffs_in_u)

--- Các hệ số của đa thức theo ẩn u, P(u) [Bessel] ---
[ 4.506244383398438  0.162495762349331  0.004218973385416
 -0.000103840850695 -0.000003630208333 -0.000000035486111
  0.000000009166667  0.000000005198413]


In [19]:
# Giả định x_coords và coeffs_in_u đã tồn tại

# --- 1. Chuẩn bị dữ liệu và chuyển đổi x -> u ---
p_coeffs_descending = coeffs_in_u[::-1] # Dùng hệ số Bessel
x_input = 1.63
n = len(x_coords) - 1
center_idx = (n + 1) // 2 - 1 # Chỉ số x0 cho Bessel
x0 = x_coords[center_idx]
h = x_coords[1] - x_coords[0]
t = (x_input - x0) / h
u_val = t - 0.5 

print("\n--- Bước 1: Chuyển đổi X sang U ---")
print(f"Giá trị x đầu vào: x = {x_input}")
print(f"Mốc x0 (Bessel): x0 = {x0}, bước nhảy h = {h}")
print(f"Giá trị t tương ứng: t = ({x_input} - {x0}) / {h} = {t}")
print(f"Giá trị u tương ứng: u = t - 0.5 = {u_val}")


# --- 2. Hàm tính đạo hàm bằng Horner (SỬA LỖI: Thêm return) ---
def tinh_bang_dao_ham_horner(coeffs_desc, u_value_to_eval):
    print("\n" + "="*50)
    print(f"Đa thức P(u) có hệ số (bậc giảm dần): {coeffs_desc}")
    print(f"Tính các đạo hàm tại u = {u_value_to_eval}")
    
    current_coeffs = list(coeffs_desc); n = len(current_coeffs) - 1
    summary_data = []; k_factorial = 1.0
    for k in range(n + 1):
        print("\n" + "="*50 + f"\n--- TÍNH TOÁN CHO ĐẠO HÀM CẤP {k} ---")
        m = len(current_coeffs); deg = m - 1
        header = [f'a_{deg-i}' for i in range(deg + 1)]; row_coeffs = current_coeffs
        row_products = ['']; row_results = []
        b = current_coeffs[0]; row_results.append(b)
        quotient_coeffs = [b]
        for i in range(1, m):
            product = b * u_value_to_eval; row_products.append(product)
            b = product + current_coeffs[i]; row_results.append(b)
            if i < m -1: quotient_coeffs.append(b)
        remainder = b
        df = pd.DataFrame([row_coeffs, row_products, row_results],
                          index=['Hệ số vào', f'Nhân với u={u_value_to_eval}', 'Kết quả'],
                          columns=header)
        display(df)
        if k > 0: k_factorial *= k
        derivative_value = remainder * k_factorial
        summary_data.append([f"P^({k})(u)", derivative_value])
        print(f"-> Số dư R_{k+1} = {remainder}")
        print(f"-> Giá trị đạo hàm d^{k}P/du^{k} = R_{k+1} * {k}! = {remainder} * {k_factorial} = {derivative_value}")
        current_coeffs = quotient_coeffs
        
    summary_df = pd.DataFrame(summary_data, columns=["Đạo hàm d^k P / du^k", f"Giá trị tại u = {u_value_to_eval}"])
    print("\n" + "="*50 + f"\n--- BẢNG TỔNG HỢP KẾT QUẢ CHO x = {x_input} ---")
    display(summary_df)
    
    # >>> SỬA LỖI Ở ĐÂY: Trả về bảng tổng hợp <<<
    return summary_df

# --- 3. Thực thi hàm (SỬA LỖI: Hứng kết quả) ---
# Gọi hàm và LƯU LẠI KẾT QUẢ vào biến summary_df_u
summary_df_u = tinh_bang_dao_ham_horner(p_coeffs_descending, u_val)

# --- 4. Tính đạo hàm theo x (SỬA LỖI: Dùng biến đã hứng) ---
print("\n" + "="*50)
print("--- TÍNH ĐẠO HÀM THEO BIẾN x (d^k P / dx^k) ---")
print(f"Sử dụng công thức: d^k P / dx^k = (1 / h^k) * (d^k P / du^k)")
derivatives_wrt_x = []

# >>> SỬA LỖI Ở ĐÂY: Dùng biến 'summary_df_u' <<<
if summary_df_u is not None:
    for k in range(len(p_coeffs_descending)):
         dkP_duk = summary_df_u.iloc[k, 1] 
         dkP_dxk = (1 / (h**k)) * dkP_duk
         derivatives_wrt_x.append([f"P^({k})(x)", dkP_dxk])
    summary_df_x = pd.DataFrame(derivatives_wrt_x, columns=["Đạo hàm d^k P / dx^k", f"Giá trị tại x = {x_input}"])
    display(summary_df_x)
else:
    print("Không thể tính đạo hàm theo x do bảng tổng hợp theo u bị thiếu.")


--- Bước 1: Chuyển đổi X sang U ---
Giá trị x đầu vào: x = 1.63
Mốc x0 (Bessel): x0 = 5.3, bước nhảy h = 0.09999999999999964
Giá trị t tương ứng: t = (1.63 - 5.3) / 0.09999999999999964 = -36.70000000000013
Giá trị u tương ứng: u = t - 0.5 = -37.20000000000013

Đa thức P(u) có hệ số (bậc giảm dần): [ 5.19841270e-09  9.16666666e-09 -3.54861111e-08 -3.63020833e-06
 -1.03840851e-04  4.21897339e-03  1.62495762e-01  4.50624438e+00]
Tính các đạo hàm tại u = -37.20000000000013

--- TÍNH TOÁN CHO ĐẠO HÀM CẤP 0 ---


,a_7,a_6,a_5,a_4,a_3,a_2,a_1,a_0
Hệ số vào,0.0,9.166667e-09,-3.548611e-08,-0.000004,-0.000104,0.004219,0.162496,4.506244
Nhân với u=-37.20000000000013,,-1.933810e-07,6.852771e-06,-0.000254,0.009569,-0.352107,12.941425,-487.465853
Kết quả,0.0,-1.842143e-07,6.817285e-06,-0.000257,0.009465,-0.347888,13.103921,-482.959608


-> Số dư R_1 = -482.9596083228876
-> Giá trị đạo hàm d^0P/du^0 = R_1 * 0! = -482.9596083228876 * 1.0 = -482.9596083228876

--- TÍNH TOÁN CHO ĐẠO HÀM CẤP 1 ---


,a_6,a_5,a_4,a_3,a_2,a_1,a_0
Hệ số vào,0.0,-1.842143e-07,0.000007,-0.000257,0.009465,-0.347888,13.103921
Nhân với u=-37.20000000000013,,-1.933810e-07,0.000014,-0.000776,0.038441,-1.782122,79.236371
Kết quả,0.0,-3.775952e-07,0.000021,-0.001033,0.047907,-2.130010,92.340292


-> Số dư R_2 = 92.34029176063204
-> Giá trị đạo hàm d^1P/du^1 = R_2 * 1! = 92.34029176063204 * 1.0 = 92.34029176063204

--- TÍNH TOÁN CHO ĐẠO HÀM CẤP 2 ---


...

,a_0
Hệ số vào,0.0
Nhân với u=-37.20000000000013,
Kết quả,0.0


-> Số dư R_8 = 5.1984126979233636e-09
-> Giá trị đạo hàm d^7P/du^7 = R_8 * 7! = 5.1984126979233636e-09 * 5040.0 = 2.619999999753375e-05

--- BẢNG TỔNG HỢP KẾT QUẢ CHO x = 1.63 ---


,Đạo hàm d^k P / du^k,Giá trị tại u = -37.20000000000013
0,P^(0)(u),-482.959608
1,P^(1)(u),92.340292
2,P^(2)(u),-15.019241
3,P^(3)(u),2.033601
4,P^(4)(u),-0.220153
5,P^(5)(u),0.017879
6,P^(6)(u),-0.000968
7,P^(7)(u),0.000026



--- TÍNH ĐẠO HÀM THEO BIẾN x (d^k P / dx^k) ---
Sử dụng công thức: d^k P / dx^k = (1 / h^k) * (d^k P / du^k)


,Đạo hàm d^k P / dx^k,Giá trị tại x = 1.63
0,P^(0)(x),-482.959608
1,P^(1)(x),923.402918
2,P^(2)(x),-1501.924078
3,P^(3)(x),2033.600863
4,P^(4)(x),-2201.530126
5,P^(5)(x),1787.852567
6,P^(6)(x),-968.040000
7,P^(7)(x),262.000000
